# ETL of wine review data and geographical data scraped from google

### Dependencies 

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import pymysql
pymysql.install_as_MySQLdb()
from config import mysql_password

## Cleanup

In [20]:
# Importing CSV files and converting to pandas dataframes

winemag_df = pd.read_csv('raw_data/winemag-data-130k-v2.csv', encoding = 'utf-8')
coordinates_df = pd.read_csv('raw_data/geo_data/winery-lat-lng-final.csv', encoding = 'utf-8')

##### Geographical data scraped from google

In [3]:
# Rounding latitude and longitude values to 5 decimal places

coordinates_df['latitude'] = round(coordinates_df['latitude'], 5)
coordinates_df['longitude'] = round(coordinates_df['longitude'], 5)

##### Downloaded wine review dataset

In [4]:
# Getting only the necessary columns
winemag_trimmed = winemag_df[['country', 'description', 'taster_name', 'points', 'price', 'province', 'title', 'variety', 'winery']]

# Drop duplicate reviews and limiting each wine to one review 
winemag_clean = winemag_trimmed.drop_duplicates(subset='description')

### Merging all the data into one dataframe

In [5]:
# Merge the two dataframes on winery
wine_merged = pd.merge(winemag_clean, coordinates_df, on='winery', how='left')

# Remove any entries with NaN values
wine_merged = wine_merged.dropna()

### Assigning an index number to each wine for database organization

In [ ]:
# Pull out titles to new df, drop duplicate titles
assigning_idx_df = wine_merged[['title']]
assigning_idx_df = assigning_idx_df.drop_duplicates(subset='title')

# Reset index to assign values only to unique titles
assigning_idx_df = assigning_idx_df.reset_index(drop=True)
assigning_idx_df['wine_id'] = assigning_idx_df.index
assigning_idx_df['wine_id'] = assigning_idx_df['wine_id'] + 1

# Use merge to bring in index numbers for all df entries
wine_df = pd.merge(wine_merged, assigning_idx_df, on='title', how='left')

In [7]:
wine_df.head(1)

,country,description,taster_name,points,price,province,title,variety,winery,latitude,longitude,wine_id
0,Portugal,"This is ripe and fruity, a wine that is smooth...",Roger Voss,87,15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,41.38769,-7.27687,1


In [8]:
# Check to see if unique titles = unique index_col 
# number of unique descriptions should be greater than unique titles
wine_df.nunique()

country           41
description    79514
taster_name       19
points            21
price            372
province         404
title          78859
variety          626
winery         11624
latitude       10768
longitude      10779
wine_id        78859
dtype: int64

### Organizing into tables for database prep

In [9]:
location = wine_df[['wine_id', 'winery', 'province', 'country', 'latitude', 'longitude']]
location = location.drop_duplicates(subset='winery')

In [10]:
review = wine_df[['wine_id', 'taster_name', 'points', 'description', 'price']]

In [11]:
bottle = wine_df[['wine_id', 'title', 'variety', 'winery']]
bottle = bottle.drop_duplicates(subset='wine_id')

### Connect to the local winery_db database

In [33]:
conn_string = "mysql+pymysql://root:" + mysql_password + "@localhost:3306/winery_db?charset=utf8mb4"
engine = create_engine(conn_string)

### Check the database for tables

In [34]:
engine.table_names()

['bottle', 'location', 'review']

### Use Pandas to load dataframes into MySQL database as tables

In [35]:
location.to_sql(name='location', con=engine, if_exists='replace', index=False)
review.to_sql(name='review', con=engine, if_exists='replace', index=False)
bottle.to_sql(name='bottle', con=engine, if_exists='replace', index=False)

### Confirm data has been loaded to database by querying the winery_db

In [36]:
pd.read_sql_query('select * from location', con=engine).head()

,wine_id,winery,province,country,latitude,longitude
0,1,Quinta dos Avidagos,Douro,Portugal,41.38769,-7.27687
1,2,Rainstorm,Oregon,US,40.08983,-88.24818
2,3,St. Julian,Michigan,US,40.01597,-105.28277
3,4,Sweet Cheeks,Oregon,US,42.34390,-71.10110
4,5,Tandem,Northern Spain,Spain,36.35816,-94.21373


In [37]:
pd.read_sql_query('select * from review', con=engine).head()

,wine_id,taster_name,points,description,price
0,1,Roger Voss,87,"This is ripe and fruity, a wine that is smooth...",15.0
1,2,Paul Gregutt,87,"Tart and snappy, the flavors of lime flesh and...",14.0
2,3,Alexander Peartree,87,"Pineapple rind, lemon pith and orange blossom ...",13.0
3,4,Paul Gregutt,87,"Much like the regular bottling from 2012, this...",65.0
4,5,Michael Schachner,87,Blackberry and raspberry aromas show a typical...,15.0


In [38]:
pd.read_sql_query('select * from bottle', con=engine).head()

,wine_id,title,variety,winery
0,1,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
1,2,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
2,3,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
3,4,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
4,5,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
